In [1]:
# !aria2c -x8 https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
# tar -xvzf UrbanSound8K.tar.gz

In [2]:

from torch.utils.data import Dataset
from tqdm import tqdm
from torchaudio.functional import resample
import torch
from torch.utils.data import DataLoader
import torchaudio.transforms as T
from torch import nn
import torch.nn.functional as F
import os
import pandas as pd
import torchaudio

In [3]:
class UrbanSound8KDataset(Dataset):
    def __init__(self, 
                 root='/workspace/postdoc/sound_datasets/urbansound8k/UrbanSound8K', # folder where the UrbanSound8K dataset is stored after extraction
                 folds=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], # this will load all folds
                 resample_freq=16000 # resample frequency
        ):
        self.root = root
        meta = pd.read_csv(os.path.join(root, 'metadata/UrbanSound8K.csv'))
        self.folded = meta[meta.fold.isin(folds)]
        self.resample_freq = resample_freq

    def __len__(self):
        return len(self.folded)

    def __getitem__(self, idx):
        row = self.folded.iloc[idx]
        file_path = os.path.join(self.root, 'audio', f'fold{row.fold}', row.slice_file_name)
        waveform, sample_rate = torchaudio.load(file_path)
        
        # single channel
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        # resample
        waveform = resample(waveform, orig_freq=sample_rate, new_freq=16000)
        # normalize
        waveform = (waveform - waveform.mean()) / (waveform.std() + 1e-7)
        
        return {'audio': waveform.squeeze(),  # move it to collate_fn to batch processing and speedup
                'class_id': row["classID"], 
                'class_label': row["class"], 
                'fold': row["fold"]}

In [4]:
def collate_fn(batch):
    waveforms = [item['audio'] for item in batch]
    batched_waveforms = torch.nn.utils.rnn.pad_sequence(waveforms, batch_first=True, padding_value=0)
    return {'audio': batched_waveforms,
            'class_id': torch.tensor([item['class_id'] for item in batch]), 
            'class_label': [item['class_label'] for item in batch], 
            'fold': torch.tensor([item['fold'] for item in batch])}

In [5]:
train_dataset = UrbanSound8KDataset(folds=[1, 2, 3, 4, 5, 6, 7, 8, 9])
val_dataset = UrbanSound8KDataset(folds=[10])

In [7]:

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn, pin_memory=True, num_workers=1)

In [8]:
class MelSpectrogram(torch.nn.Module):
    def __init__(
        self,
        n_fft: int = 400,
        win_length: int = 400,  # 25ms
        hop_length: int = 160,  # 10ms
        n_mels: int = 80,
        sample_rate: int = 16000,
    ):
        super().__init__()
        self.spectogram = T.Spectrogram(n_fft, win_length, hop_length, power=2)
        self.mel_scale = T.MelScale(n_mels, sample_rate, n_stft=n_fft // 2 + 1, mel_scale="slaney", norm="slaney")

    def forward(self, waveforms):
        spec = self.spectogram(waveforms)
        mel_spec = self.mel_scale(spec.float())        
        # the following is doing AmplitudeToDB conversion, but it does not match torchaudios implementation
        log_spec = torch.log10(torch.clamp(mel_spec, min=1e-10))
        log_spec = torch.max(log_spec, log_spec.max() - 8.0)
        log_spec = (log_spec + 4.0) / 4.0
        return log_spec
    
mel_spectrogram_fn = MelSpectrogram()

In [9]:
# a simple audio classifier with 92K params, we can use more sophisticated models later

class AudioClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.convnet = nn.Sequential(
            # 1-D conv across the time dimension
            nn.Conv2d(1, 32, kernel_size=(1, 5)),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(1, 3)),
            nn.Dropout(0.1),

            nn.Conv2d(32, 32, kernel_size=(5, 5)),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),
            nn.Dropout(0.1),

            nn.Conv2d(32, 32, kernel_size=(5, 5)),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),
            nn.Dropout(0.1),

            nn.Conv2d(32, 32, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),
            nn.Dropout(0.1),

            nn.Conv2d(32, 32, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),
            nn.Dropout(0.1),

            nn.Flatten(),

            nn.Linear(160, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),

            nn.Linear(128, num_classes),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], 1, x.shape[1], x.shape[2])
        x = self.convnet(x)
        return x

def num_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

clf = AudioClassifier(num_classes=10)
print(f"Total number of parameters: {num_parameters(clf) / 1e3}k")

Total number of parameters: 92.426k


In [10]:
optim = torch.optim.AdamW(clf.parameters(), lr=1e-4, weight_decay=1e-3)

# select gpu 0
device = torch.device("cuda:0")
clf.to(device)
mel_spectrogram_fn.to(device)

for epoch in range(40):
    clf.train()
    ys = []
    y_hats = []
    progress_bar = tqdm(train_loader, desc=f"Training epoch  {epoch}")
    for batch in progress_bar:
        optim.zero_grad()
        x = batch['audio'].to(device)
        y = batch['class_id'].to(device)
        ys.append(y)
        with torch.no_grad():
            spec = mel_spectrogram_fn(x)
        y_hat = clf(spec)
        y_hats.append(y_hat)
        loss = F.nll_loss(y_hat, y)
        progress_bar.update()
        progress_bar.set_postfix(loss=loss.item())
        loss.backward()
        optim.step()
    ys = torch.cat(ys)
    y_hats = torch.cat(y_hats)
    acc = (ys == y_hats.argmax(dim=1)).float().mean()
    print(f"Training accuracy:  {acc.item() * 100:.2f}%")
    
    # evaluation
    clf.eval()
    ys = []
    y_hats = []
    progress_bar = tqdm(val_loader, desc=f"Validating epoch {epoch}")
    for batch in progress_bar:
        x = batch['audio'].to(device)
        y = batch['class_id'].to(device)
        ys.append(y)
        with torch.no_grad():
            spec = mel_spectrogram_fn(x)
            y_hat = clf(spec)
        y_hats.append(y_hat)
        progress_bar.update()
    ys = torch.cat(ys)
    y_hats = torch.cat(y_hats)
    acc = (ys == y_hats.argmax(dim=1)).float().mean()
    print(f"Validation accuracy: {acc.item() * 100:.2f}%")


Training epoch  0:   0%|          | 0/247 [00:00<?, ?it/s]

Training epoch  0: 100%|██████████| 247/247 [00:47<00:00,  5.18it/s, loss=1.7] 


Training accuracy: 26.79%


Validating epoch 0: 100%|██████████| 27/27 [00:04<00:00,  5.41it/s]


Validation accuracy: 30.82%


...

Validation accuracy: 71.45%


Training epoch  39: 100%|██████████| 247/247 [00:46<00:00,  5.29it/s, loss=0.196]


Training accuracy: 89.68%


Validating epoch 39: 100%|██████████| 27/27 [00:04<00:00,  5.43it/s]

Validation accuracy: 75.15%
